In [36]:
import json
import pandas as pd

In [17]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/TikTok/Input_test/HJ_tiktol_json_strcure.json"

with open(main_path, 'r') as file:
    data = json.load(file)




In [ ]:
max_columns = 7

with open(main_path, 'r') as file:
    data = json.load(file)

dict_list = [data]
for i in range(1, max_columns):
    print(i)
    dict_holder = []
    for d in dict_list:
        if isinstance(d, dict):
            for k, v in d.items():
                if isinstance(v, list):
                    v = v[0]
                    print(k)
                    #print(v)
                    dict_holder.append(v)
                    #print(dict_holder)
                    
                       
                else:
                    print(k)
                    #print(v)
                    dict_holder.append(v)
                    #print(dict_holder)
        else:
            continue
    dict_list = dict_holder
    #print(dict_list)
                
            


        



1
user_data_tiktok.json
2
Ads and data
App Settings
Comment
Direct Message
Income Plus Wallet Transactions
Location Review
Post
Profile
TikTok Shop
Tiktok Live
Your Activity
3
Ad Interests
Instant Form Ads Responses
Off TikTok Activity
Block List
Settings
Comments
Direct Messages
Transaction History
Location Reviews
Posts
Recently Deleted Posts
AI-Moji
Autofill
Profile Info
Communication With Shops
Current Payment Information
Customer Support History
Order Dispute History
Order History
Product Browsing History
Product Reviews
Returns and Refunds History
Saved Address Information
Shopping Cart List
Vouchers
Go Live History
Go Live Settings
Watch Live History
Watch Live Settings
Activity Summary
Favorite Effects
Favorite Hashtags
Favorite Sounds
Favorite Videos
Follower
Following
Hashtag
Like List
Login History
Most Recent Location Data
Purchases
Searches
Share History
Status
Watch History
4
AdInterestCategories
ResponsesList
OffTikTokActivityDataList
App
BlockList
App
SettingsMap
App
Co

In [ ]:
max_columns = 7

with open(main_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame()
df['column_0_values'] = data
df.reset_index(inplace = True)


for i in range(max_columns):
    df[f'column_{i}'] = ''
    df[f'column_{i+1}_values'] = ''
    print(i)

    for i, row in df.iterrows():
        value= row[f'column_{i}_values']
        if isinstance(value, dict):
            key_list = []
            value_list = []
            for k, v in value.items():
                if isinstance(v, list):
                    v = v[0]
                    k
                    value_list.append(v)
                       
                       
                else:
                    k
                    value_list.append(v)  
                
                print(value_list)


            df.at[i, f'column_{i}'] = k
            df.at[i, f'column_{i}_values'] = value_list
            df.explode(f'column_{i}_values')


        else:
            continue
    
        dict_list = dict_holder
    #print(dict_list)



0
[{'Ad Interests': {'AdInterestCategories': 'string'}, 'Instant Form Ads Responses': {'ResponsesList': 'object'}, 'Off TikTok Activity': {'OffTikTokActivityDataList': 'object'}}]
[{'Ad Interests': {'AdInterestCategories': 'string'}, 'Instant Form Ads Responses': {'ResponsesList': 'object'}, 'Off TikTok Activity': {'OffTikTokActivityDataList': 'object'}}, {'Block List': {'App': 'number', 'BlockList': 'object'}, 'Settings': {'App': 'number', 'SettingsMap': {'Allow DownLoad': 'string', 'Allow Others to Find Me': 'string', 'App Language': 'string', 'Content Preferences': {'Keyword filters for videos in Following feed': 'array', 'Keyword filters for videos in For You feed': 'array', 'Video Languages Preferences': 'array'}, 'Family Content Preferences': {}, 'Filter Comments': 'string', 'Interests': 'string', 'Personalized Ads': 'string', 'Private Account': 'string', 'Push Notification': {'Desktop notification': 'string', 'New Comments on My Video': 'string', 'New Fans': 'string', 'New Likes

In [46]:
print(data)

{'user_data_tiktok.json': {'Ads and data': {'Ad Interests': {'AdInterestCategories': 'string'}, 'Instant Form Ads Responses': {'ResponsesList': 'object'}, 'Off TikTok Activity': {'OffTikTokActivityDataList': 'object'}}, 'App Settings': {'Block List': {'App': 'number', 'BlockList': 'object'}, 'Settings': {'App': 'number', 'SettingsMap': {'Allow DownLoad': 'string', 'Allow Others to Find Me': 'string', 'App Language': 'string', 'Content Preferences': {'Keyword filters for videos in Following feed': 'array', 'Keyword filters for videos in For You feed': 'array', 'Video Languages Preferences': 'array'}, 'Family Content Preferences': {}, 'Filter Comments': 'string', 'Interests': 'string', 'Personalized Ads': 'string', 'Private Account': 'string', 'Push Notification': {'Desktop notification': 'string', 'New Comments on My Video': 'string', 'New Fans': 'string', 'New Likes on My Video': 'string'}, 'Suggest your account to Facebook friends': 'string', 'Suggest your account to contacts': 'strin